<a href="https://colab.research.google.com/github/jqllxew/ChatGLM-6B/blob/main/chat_glm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import ipywidgets as widgets
from IPython.utils import capture
import os

def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
with capture.capture_output() as cap:
  !pip install transformers==4.27.1 protobuf cpm_kernels gradio mdtex2html sentencepiece
!mkdir -p /content/gdrive/MyDrive/cache/transformers
os.environ['TRANSFORMERS_CACHE']=f"/content/gdrive/MyDrive/cache/transformers"
!ln -s /content/gdrive/MyDrive/cache/ /root/.cache/
inf('\u2714 Done','success', '50px')

In [ ]:
from transformers import AutoModel, AutoTokenizer
import gradio as gr
import mdtex2html
import time
import torch

tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm-6b-int4", trust_remote_code=True).half().cuda()
model = model.eval()

def postprocess(self, y):
    if y is None:
        return []
    for i, (message, response) in enumerate(y):
        y[i] = (
            None if message is None else mdtex2html.convert(message),
            None if response is None else mdtex2html.convert(response),
        )
    return y


gr.Chatbot.postprocess = postprocess


def parse_text(text):
  """copy from https://github.com/GaiZhenbiao/ChuanhuChatGPT/"""
  lines = text.split("\n")
  lines = [line for line in lines if line != ""]
  count = 0
  for i, line in enumerate(lines):
    if "```" in line:
      count += 1
      items = line.split('`')
      if count % 2 == 1:
        lines[i] = f'<pre><code class="language-{items[-1]}">'
      else:
        lines[i] = f'<br></code></pre>'
    else:
      if i > 0:
        if count % 2 == 1:
          line = line.replace("`", "\`")
          line = line.replace("<", "&lt;")
          line = line.replace(">", "&gt;")
          line = line.replace(" ", "&nbsp;")
          line = line.replace("*", "&ast;")
          line = line.replace("_", "&lowbar;")
          line = line.replace("-", "&#45;")
          line = line.replace(".", "&#46;")
          line = line.replace("!", "&#33;")
          line = line.replace("(", "&#40;")
          line = line.replace(")", "&#41;")
          line = line.replace("$", "&#36;")
        lines[i] = "<br>"+line
  text = "".join(lines)
  return text


def predict(input, chatbot, max_length, top_p, temperature, history):
  chatbot.append((parse_text(input), ""))
  for response, history in model.stream_chat(
    tokenizer, input, history, max_length=max_length, top_p=top_p, temperature=temperature):
    chatbot[-1] = (parse_text(input), parse_text(response))
    yield chatbot, history

def reset_user_input():
  return gr.update(value='')

def reset_state():
  return [], []
inf('\u2714 Done','success', '50px')

In [ ]:
with gr.Blocks() as demo:
  gr.HTML("""<h1 align="center">ChatGLM</h1>""")
  chatbot = gr.Chatbot()
  with gr.Row():
    with gr.Column(scale=4):
      with gr.Column(scale=12):
        user_input = gr.Textbox(show_label=False, placeholder="Input...", lines=10).style(container=False)
      with gr.Column(min_width=32, scale=1):
        submitBtn = gr.Button("Submit", variant="primary")
    with gr.Column(scale=1):
      emptyBtn = gr.Button("Clear History")
      max_length = gr.Slider(0, 4096, value=2048, step=1.0, label="Maximum length", interactive=True)
      top_p = gr.Slider(0, 1, value=0.7, step=0.01, label="Top P", interactive=True)
      temperature = gr.Slider(0, 1, value=0.95, step=0.01, label="Temperature", interactive=True)
  history = gr.State([])
  submitBtn.click(predict, [user_input, chatbot, max_length, top_p, temperature, history], [chatbot, history], show_progress=True)
  submitBtn.click(reset_user_input, [], [user_input])
  emptyBtn.click(reset_state, outputs=[chatbot, history], show_progress=True)
demo.queue().launch(share=True, inbrowser=False)
try:
  while 1:
    time.sleep(1)
except KeyboardInterrupt as e:
  pass
finally:
  demo.close()
  torch.cuda.empty_cache()
  inf('\u2714 Close','success', '50px')